In [1]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import copy
import math
import random
import time


# Loading trace : Needs to expanded into 4K chunks
path = r'/Users/chandranilchakraborttii/Documents/GC_pred/data'
all_files = glob.glob(os.path.join(path, "2016030917-LUN2.csv_dataprep_deathtime_added.csv"))

f = all_files[0]  # Change the file name as required
print("File " + str(f))
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
cols = ['IO_Num','LBA','deathtime']
df.columns = cols
print("Before",len(df))
df['deathtime'] = df['deathtime'].replace(np.NaN, -1)
df = df.loc[df['deathtime'] != -1]
print("After",len(df))
lba_list = df['LBA'].tolist()
deathtime_list = df['deathtime'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

File /Users/chandranilchakraborttii/Documents/GC_pred/data/2016030917-LUN2.csv_dataprep_deathtime_added.csv
Before 1630000
After 943841
Min LBA in the dataset : 658733056
Max LBA in the dataset : 5335229436416
Number of unique LBAs in the data : 52954
Number of IO Accesses : 943841


In [13]:
#SSD specifications
num_page_addresses = len(Counter(df['LBA']))
page_size = 4096
page_per_block = 64    
GB = 1024*1024*1024
SSD_size = num_page_addresses*page_size
# SSD_size_GB_normal = round_decimals_up(SSD_size/GB,1)
SSD_size_GB_normal = SSD_size/GB
over_provisioning_ratio = 0.2
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))
# SSD_size_full = round_decimals_up((1 + over_provisioning_ratio)*SSD_size_GB_normal,1)
SSD_size_full = (1 + over_provisioning_ratio)*SSD_size_GB_normal
print("SSD Capacity (Available in GB) :",SSD_size_GB_normal)
print("SSD Capacity (Total in GB)     :",SSD_size_full)

SSD Capacity (Available in GB) : 0.20200347900390625
SSD Capacity (Total in GB)     : 0.24240417480468748


In [14]:
GB = 1024*1024*1024
ssd_capacity = SSD_size_full *GB

# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

block_struct = {}
for x in free_list_block:
    start_lba = x
    write_ptr=0
    invalid_pages=0
    death_time = 0
    valid_bitmap = []
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)

    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    block_struct[start_lba]=segment
    

print("Total number of Blocks created: ", len(block_addresses))
print("Total number of Pages created:  ", len(page_addresses))
print(str(len(block_struct)) + " 4K blocks Initialized" )

Total number of Blocks created:  929
Total number of Pages created:   59450
929 4K blocks Initialized


In [15]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                          # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[prev_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)


    
#map LBA to phys
def map_lba(lba,deathtime, block_IO_burst):
    # Finding which block to add the LBA
    for x in block_IO_burst:
        if(block_struct[x][3] == page_per_block):
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            
    block_map={}
    found = False
    for x in block_IO_burst:
        # If the death Time has passed, make it priority 
        if(block_struct[x][5] == True):
            block_select = x
            found = True 
        else:
            block_map[x]= block_struct[x][4]
            
    # Finding the block with closest death time 
    if(found !=True):
        delta = max(deathtime_range_list)*100
        keys = list(block_map.keys())
        block_select = -1
        for x in keys:
            tmp = abs(block_map[x] - deathtime)
            if(tmp < delta):
                delta = tmp
                block_select = x


        
    # Block Found, now updating block          
    phys_addr = block_struct[block_select][0] + (block_struct[block_select][3])
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[block_select][2][block_struct[block_select][3]] = True             # Setting Bitmap
    block_struct[block_select][1] = block_struct[block_select][2].count(True)            # Setting invalid pages
    block_struct[block_select][3] = block_struct[block_select][3] + 1               # Increasing Write pointer
    total_user_writes[0] = total_user_writes[0] + 1

    

#check if we need to close/open block. Do not perform GC if we are already
def check_GC (block_IO_burst, in_gc):
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] == page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            
            write_ptr=0
            invalid_pages=0
            valid_bitmap = []
            priority = False
           
            for x in range(page_per_block):
                valid_bitmap.append(False)

            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][1] = invalid_pages
            block_struct[new_block][2] = valid_bitmap
            block_struct[new_block][3] = write_ptr
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    if(len(free_list_block) == 0):
        print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
   # Checking if GC is needed
    elif (len(free_list_block) <= GC_THRESHOLD):
        # Checking if GC is already going on
        if(in_gc != True):
            in_gc = do_greedy_gc(block_IO_burst,in_gc) 
    return block_IO_burst




def do_greedy_gc(block_IO_burst,in_gc):
    in_gc = True
    if(len(closed_blocks)):
        total_user_writes[1]=1
    min_val = float('inf')
    
    for x in closed_blocks:              
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x
        # For each closed block, check phys_addr: If valid bitmap is True (data is valid), copy to OP capacity
#     print(min_val)
#     print(block_struct[gc_blk])
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            gc_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            total_gc_writes[0] = total_gc_writes[0] + 1
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,death_time,block_IO_burst)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    
                
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
        
    death_time = block_struct[prev_block][6]                      # Getting current death_time for the block
    free_list_block.append(gc_blk)
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] >= page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
            block_struct[new_block][5] = False
    
    in_gc = False
    return in_gc

def decrease_death_time(block_IO_burst):
    # Decreasing Death Time for each block
    for x in block_IO_burst:
        # If death time passed, make the block priority 
        # Priority means: Join with blocks with nearest death time until full
        if (block_struct[x][4] <= 0):
            block_struct[x][5] = True
            merge(block_IO_burst)
        else:
            block_struct[x][4] = block_struct[x][4] - 1  
    return block_IO_burst



# Find candidate block (whose death time has passed)
# Find target blocks whose death time is most recent to expire
# If first target block is full, request a new block
# Fill the next blocks until the candidate block is empty
# Request a new block for candidate block

def merge(block_IO_burst):
    diff = float('inf')
    for x in block_IO_burst:
        if block_struct[x][5] <= 0:
            candidate = x
            break
    
    block_IO_burst.remove(candidate)
    death_time_candidate = block_struct[x][6]
    # merge the blocks and request a new block
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the target block
        phys_addr = block_struct[candidate][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            target_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        death_time = block_struct[prev_block][4]                      # Getting current death_time for the block
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(target_lba)
            priority_writes[0] = priority_writes[0] + 1
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block to a new location
            map_lba(target_lba,death_time,block_IO_burst)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    
    
    while (len(block_IO_burst) < num_cur_blocks_open):
        new_block = free_list_block.pop(0)
        write_ptr=0
        invalid_pages=0
        valid_bitmap = []
        priority = False

        for x in range(page_per_block):
            valid_bitmap.append(False)

        block_IO_burst.append(new_block)              # Adding to open blocks
        block_struct[new_block][1] = invalid_pages
        block_struct[new_block][2] = valid_bitmap
        block_struct[new_block][3] = write_ptr
        block_struct[new_block][4] = death_time_candidate       # Setting death time counter of the new block
        block_struct[new_block][5] = False
        block_struct[new_block][6] = death_time_candidate       # Setting death time of the new block
        block_IO_burst.append(block_num) 
    
   
    

In [16]:
# Setting global parameters
# Initalizing Starting Free Blocks..
global gc_writes
global in_gc 

num_cur_blocks_open = 20        # Hyperparameter

L2P = {}
P2L = {}
closed_blocks = []
cur_blocks_open = []
lba_burst = []
deathtime_range_list = []
interval = float(100/num_cur_blocks_open)
gc_writes = 0
in_gc = False
death_time_passed = []

for x in range(num_cur_blocks_open):
    deathtime_range_list.append(int(np.percentile(deathtime_list, (x+1)*interval)))


block_IO_burst = []
death_time_ranges = []
print("Initalizing Starting Free Blocks...")
for x in range(num_cur_blocks_open):
    block_num = free_list_block.pop(0)                                            # Getting a free block
    block_IO_burst.append(block_num) 
    block_struct[block_num][4] = deathtime_range_list[x]                          # Setting death time
    block_struct[block_num][6] = deathtime_range_list[x]  

Initalizing Starting Free Blocks...


In [17]:
total_gc_writes = []
total_gc_writes.append(0)
priority_writes = []
priority_writes.append(0)
total_user_writes = []
total_user_writes.append(0)
total_user_writes.append(0)
blocks_closed_early = []
counter = 0
GC_THRESHOLD = 100

print("Starting Trace..!")
start_time = time.time()
while(counter < len(lba_list)):
    if(counter >100000 and counter%100000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    lba= int(lba_list[counter])
    death_time = int(deathtime_list[counter])
    if lba in L2P:
        invalidate_lba(lba)
    block_IO_burst = check_GC(block_IO_burst,in_gc)
    map_lba(lba,death_time,block_IO_burst)
    block_IO_burst = decrease_death_time(block_IO_burst)
    counter = counter + 1

end_time = time.time()
run_time = end_time - start_time
print("Execution Time for the FTL :",run_time)
print("Total Number of writes :",total_user_writes[0])
print("Total Number of GC writes :",total_gc_writes[0])
print("Total Number of Priority writes    :",priority_writes[0])
print("Total Number of user writes :",(total_user_writes[0] - total_gc_writes[0]))

Starting Trace..!
Percentage completed in (%)  : 21.19000975799949
Percentage completed in (%)  : 31.785014636999243
Percentage completed in (%)  : 42.38001951599898
Percentage completed in (%)  : 52.97502439499874
Percentage completed in (%)  : 63.570029273998486
Percentage completed in (%)  : 74.16503415299823
Percentage completed in (%)  : 84.76003903199796
Percentage completed in (%)  : 95.35504391099772
Execution Time for the FTL : 18.475670099258423
Total Number of writes : 944066
Total Number of GC writes : 0
Total Number of Priority writes    : 225
Total Number of user writes : 944066


In [23]:
print("GC Threshold :",GC_THRESHOLD)
print("OverProvisioning Ratio :",over_provisioning_ratio)
print("Number of Open Blocks :",num_cur_blocks_open)
print("Total Number of writes :",total_user_writes[0])
print("Total Number of GC writes :",total_gc_writes[0])
print("Total Number of Priority writes    :",priority_writes[0])
print("Write Amplification:  ",total_user_writes[0]/(total_user_writes[0] - total_gc_writes[0] - priority_writes[0]))
print("Priority writes ratio to total writes",priority_writes[0]/total_user_writes[0])
print("Execution Time for the FTL :",run_time)
print("Total Number of user writes :",(total_user_writes[0] - total_gc_writes[0] - priority_writes[0]))
print("Done")

GC Threshold : 100
OverProvisioning Ratio : 0.2
Number of Open Blocks : 20
Total Number of writes : 944066
Total Number of GC writes : 0
Total Number of Priority writes    : 225
Write Amplification:   1.0002383876097776
Priority writes ratio to total writes 0.0002383307946690168
Execution Time for the FTL : 18.475670099258423
Total Number of user writes : 943841
Done


In [19]:
print("Done")

Done
